## Exercice individuel: création de prédiction sur un seul input

Importer les données

In [1]:
from pandas import read_csv
from bigml.api import BigML
import pandas as pd
df = read_csv('../handson-ml2/exo.csv', index_col=0)
trainfull = read_csv('trainfull.csv', index_col=0)

In [2]:
api = BigML(project= 'project/5d94a407eba31d45c8000088')

In [3]:
# Intitialise lists data. 
data = [{'Id': 1,
         'RevolvingUtilizationOfUnsecuredLines': 0.01703559, 
         'age': 42, 
         'NumberOfTime30-59DaysPastDueNotWorse': 1, 
         'DebtRatio': 0, 
         'MonthlyIncome': 0, 
         'NumberOfOpenCreditLinesAndLoans': 6, 
         'NumberOfTimes90DaysLate': 0, 
         'NumberRealEstateLoansOrLines': 1, 
         'NumberOfTime60-89DaysPastDueNotWorse': 0, 
         'NumberOfDependents': 1,
         'SeriousDlqin2yrs': 0,
         'IncomePerPerson': 0, 
         'NumberOfDaysLate': 1, 
         'NumberCreditLines': 5, 
         'MonthlyDebt': 0, 
         'MonthlyBalance': 0}]

In [4]:
df_test = pd.DataFrame(data, columns = ['Id',
 'RevolvingUtilizationOfUnsecuredLines',
 'age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'MonthlyIncome',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberOfTimes90DaysLate',
 'NumberRealEstateLoansOrLines',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfDependents',
 'SeriousDlqin2yrs',
 'IncomePerPerson',
 'NumberOfDaysLate',
 'NumberCreditLines',
 'MonthlyDebt',
 'MonthlyBalance'])
df_test

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,IncomePerPerson,NumberOfDaysLate,NumberCreditLines,MonthlyDebt,MonthlyBalance
0,1,0.017036,42,1,0,0,6,0,1,0,1,0,0,1,5,0,0


In [5]:
df_test['NumberOfDaysLate'] = df_test['NumberOfTimes90DaysLate']+ df_test['NumberOfTime60-89DaysPastDueNotWorse']+ df_test['NumberOfTime30-59DaysPastDueNotWorse']
df_test['NumberCreditLines'] = df_test['NumberOfOpenCreditLinesAndLoans']- df_test['NumberRealEstateLoansOrLines']
df_test

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,IncomePerPerson,NumberOfDaysLate,NumberCreditLines,MonthlyDebt,MonthlyBalance
0,1,0.017036,42,1,0,0,6,0,1,0,1,0,0,1,5,0,0


In [6]:
df_test.to_csv("exotest.csv")

Préparation de l'array d'output pour le train

In [7]:
target_column = 'SeriousDlqin2yrs'
y_train = trainfull[target_column].values
print(y_train)

[1 0 0 ... 0 0 0]


Préparation de l'array d'input pour le train

In [8]:
from pprint import pprint
X_train = trainfull.drop(target_column, axis=1).values
pprint(X_train)

array([[1.00000000e+00, 7.66126609e-01, 4.50000000e+01, ...,
        7.00000000e+00, 7.32319702e+03, 1.79680298e+03],
       [2.00000000e+00, 9.57151019e-01, 4.00000000e+01, ...,
        4.00000000e+00, 3.16878123e+02, 2.28312188e+03],
       [3.00000000e+00, 6.58180140e-01, 3.80000000e+01, ...,
        2.00000000e+00, 2.58914887e+02, 2.78308511e+03],
       ...,
       [1.49998000e+05, 2.46043918e-01, 5.80000000e+01, ...,
        1.70000000e+01, 0.00000000e+00, 0.00000000e+00],
       [1.49999000e+05, 0.00000000e+00, 3.00000000e+01, ...,
        4.00000000e+00, 0.00000000e+00, 5.71600000e+03],
       [1.50000000e+05, 8.50282951e-01, 6.40000000e+01, ...,
        6.00000000e+00, 2.03875009e+03, 6.11924991e+03]])


Même chose pour le test

In [9]:
X_test = df_test.drop(target_column, axis=1).values
y_test = df_test[target_column].values
pprint(X_test)

array([[1.000000e+00, 1.703559e-02, 4.200000e+01, 1.000000e+00,
        0.000000e+00, 0.000000e+00, 6.000000e+00, 0.000000e+00,
        1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00,
        1.000000e+00, 5.000000e+00, 0.000000e+00, 0.000000e+00]])


In [12]:
import pickle
from xgboost import XGBClassifier
filename = 'finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb')) # Récupérer le modèle entrainé
result = loaded_model.score(X_test, y_test)
print(result)
y_test_proba = loaded_model.predict_proba(X_test)[:,1]
print(y_test_proba)

1.0
[0.0449454]


Probabilité de défaut de remboursement = 4,5%, par contre bizarrement le modèle aurait prédit un 1 bien que la probabilité de 1 soit très petite.